# Patch Basics


This notebook shows how to work with DASCore's [`Patch`](https://dascore.org/api/dascore/core/patch/Patch.html). It is a shortened version of the [DASCore's Patch tutorial](https://dascore.org/tutorial/patch.html). 

In [ ]:
%%capture

# First ensure DASCore is installed. If not, install and restart the kernel.
try:
    import dascore as dc
except ImportError:
    !pip install dascore
    !pip install ipympl
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel


import numpy as np
from rich import print

# [Patch](https://dascore.org/tutorial/patch.html) 

To demonstrate the patch, we use an example seismic event recorded shown in [this study](https://www.frontiersin.org/journals/earth-science/articles/10.3389/feart.2022.907749/full).

The `Patch` is composed of:
- `coords`: the coordinates of each dimension (e.g. coord range, step, units)
- `data`: the array of measurements
- `attrs`: the non-coordinate metadata (e.g. cable id)
- `dims`: a tuple of dimension names (e.g. distance, frequency)

The `Patch` object is *immutable* which means that once a patch is created it cannot (easily) be changed. This makes it safe to share compontents between patches and simplifies parallel processing. 

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")


In [ ]:
# Display the string representation of the patch object:
print(patch)

## `Patch` Components: Data and Dims

The data array can be accessed with `data` and updated with [`Patch.update`](https://dascore.org/tutorial/patch.html#update)

In [ ]:
# Get the data array
array = patch.data

In [ ]:
# Divide the data by 1_000_000 and create a new patch.
patch_new_data = patch.update(data=array / 1_000_000)

The dimensions are accessed via the `dims` attribute.

In [ ]:
print(patch.dims)

### **Exercise** (`Patch` Data)

Calculate and print following:

1. The number of samples in the data
2. The minimum and maximum value of the data

In [ ]:
# Exercise (Patch Data)


## Patch Components: Attrs
Attrs contain non-dimensional metdata. Acquisition/interrogator identifiers, custom tags, etc.


In [ ]:
print(patch.attrs)

Notice the `acquisition_id` and `cable_id` are not set. We could set or update `Attrs` like so: 

In [ ]:
patch_updated_attrs = patch.update_attrs(
    acquisition_id="experiment_12",
    cable_id="b202393ad",
)
print(patch_updated_attrs)

### **Exercise** (`Patch` Attrs)
Do the following:

1. Print the `cable_id` from the patch with updated attrs.
2. Create a new patch with a `station name` of "DAS1".

In [ ]:
# Exercise (Patch Attrs)


## Patch Components: Coords

`Coords` represent information about the coordinates (not coordinate systems) associated with a patch. These include, but aren't limited to, the dimensions such as 'time' and 'distance'.


In [ ]:
print(patch.coords) # The coordinates and their labels. 

`Coords` objects can be accessed with [`Patch.get_coord`](https://dascore.org/api/dascore/proc/coords/get_coord.html). 

In [ ]:
dist_coord = patch.get_coord("distance")

# Get the start, stop, step, and units of the distance coord
dist_start = dist_coord.min()
dist_stop = dist_coord.max()
dist_step = dist_coord.step
dist_units = dist_coord.units

# Get the underlying array data in the coordinate.
dist_array = dist_coord.values

Alternatively, [`Patch.get_array`](https://dascore.org/api/dascore/proc/coords/get_array.html) simply returns the numpy array of the coordinate. 

In [ ]:
# The values of the distance dimension
dist = patch.get_array("distance")

# The value of the time dimension using a numpy datetime64 array.
time = patch.get_array("time")

# Convert the array to seconds from 1970 (floats)
time_s = dc.to_float(time)
# time_s = np.float(time)

Coordinates can be renamed using [`Patch.rename_coord`](https://dascore.org/api/dascore/proc/coords/rename_coords.html)

In [ ]:
patch_renamed_coords = patch.rename_coords(distance="depth")
print(patch_renamed_coords.dims)

Or updated using [`Patch.update_coords`](https://dascore.org/api/dascore/proc/coords/update_coords.html)

In [ ]:
from dascore.units import m

# Add 12 m to the current distance values.
dist = patch.get_array("distance")
patch_new_dist = patch.update_coords(distance=(dist + 12)*m)

dist2 = patch_new_dist.get_coord("distance")

In [ ]:
print(dist2)

### **Exercise** (`Patch` Coords)

Calculate the and print following parameters:

1. The duration (time) of the patch recording using the time coordinate.
2. Reset the start of the time coordiante (e.g., 08:00, April 14, 2025).


In [ ]:
# Exercise (Patch Coords)


## `Patch` [Visualization](https://dascore.org/tutorial/visualization.html)
DASCore provides visualization in the `patch.viz` namespace.


In [ ]:
# The classic waterfall plot
patch.viz.waterfall();

In [ ]:
# Adjust the scale parameter to make the event more visible
# Try different values to see what works best
patch.viz.waterfall(scale=0.03);  # Smaller scale value increases contrast

In [ ]:
# A wiggle plot
patch.viz.wiggle();

In [ ]:
# A wiggle plot
patch.viz.wiggle(scale=0.1); # Smaller scale value increases contrast

## Trimming and sub-selection
[`Patch.select`](https://dascore.org/api/dascore/core/spool/DataFrameSpool/select.html) is used to trim patches. For example, to zoom in on the down-going reflection in our example patch. 

In [ ]:
trimmed = patch.select(time=(.16, .22), distance =(600,800))

In [ ]:
trimmed.viz.waterfall(scale=0.1);

Trimming can also be done via samples

In [ ]:
# Remove 20 samples from start and end of time dimension using samples=True
trimmed_patch = patch.select(time=(20, -20), samples=True)
trimmed_patch.viz.waterfall(scale=0.05);

In [ ]:
# Remove 0.02 seconds from start and 0.03 seconds from the end of patch using relative=True
trimmed_patch = patch.select(time=(0.02, -0.03), relative=True)
trimmed_patch.viz.waterfall(scale=0.1);

### **Exercise** (`Patch` Select)
Remove the first 10 spatial channels then the last .05 seconds. 

In [ ]:
# Exercise (Patch Select)


## `Patch` [Processing](https://dascore.org/tutorial/processing.html)
DASCore provides many [processing](https://dascore.org/api/dascore/proc.html) patch methods. This section will highlight a few of these.


### [Pass Filtering](https://dascore.org/api/dascore/proc/filter/pass_filter.html)
The [`Patch.pass_filter`](https://dascore.org/api/dascore/proc/filter/pass_filter.html) method is used to apply bandpass, lowpass, and highpass SOS filters to the data along a specified dimension.

In [ ]:
# apply a 100Hz to 300Hz highpass
patch_bp = patch.pass_filter(time=(100, 300))  

# apply a 300Hz lowpass
patch_lp = patch.pass_filter(time=(..., 300))  

# apply a 50Hz highpass
patch_hp = patch.pass_filter(time=(50, ...))  

### Units (time)

In [ ]:
from dascore.units import Hz, s

patch = dc.get_example_patch()

# Band-pass filter between 10Hz and 40Hz
patch_filt = patch.pass_filter(time=(10*Hz, 40*Hz))

# Band-pass filter between 20s and 2s
patch_filt = patch.pass_filter(time=(2*s, 20*s))

### [Frequency-wavenumber (F-K) filter](https://dascore.org/api/dascore/proc/filter/slope_filter.html) 
The [`Patch.slope_filter`](https://dascore.org/api/dascore/proc/filter/slope_filter.html) method is used as an F-K (frequency wavenumber) filter to attenuate energy with specified apparent velocities.

In [ ]:
import matplotlib.pyplot as plt

patch = dc.get_example_patch("example_event_1")


patch_slope_fil = (
    patch.set_units("1/s", distance="m", time="s")
    .detrend("time")
    .taper(time=0.05)
    .slope_filter(filt=[2e3,2.2e3,8e3,2e4],
                directional=True,
                notch=False)
)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

patch.viz.waterfall(ax=axs[0],scale=0.05);
axs[0].set_title('Before F-K filter');

patch_slope_fil.viz.waterfall(ax=axs[1],scale=0.05);
axs[1].set_title('After F-K filter');

### **Exercise** (`Patch` processing 1)
Plot each of the pass-filtered patches. Which filtering technique did the best at enhancing the event signal? 

In [ ]:
# Exercise (Patch processing 1)


### Decimate
`decimate` reduces the number of samples in the specified dimension. By default, a lowpass filter is applied to avoid aliasing.

In [ ]:
# keep every 10th sample along time axis
patch_dec = patch.decimate(time=10)  

### Detrend

`Patch.detrend` applies a linear detrend along a specified dimension (axis).

In [ ]:
patch_detrended = patch.detrend("time")

### Resample

Like `decimate`, `resample` is used to change the sampling rate of the patch along a specific dimension. Unlike `decimate`, however, non-integar multiples can be used.

In [ ]:
patch_resamp = patch.resample(distance=15)  # change spatial sampling to 15m

### Processing methods are chained together

In [ ]:
patch = dc.get_example_patch("example_event_1")

patch_proc = (
    patch.set_units("1/s", distance="m", time="s")
    .detrend("time")
    .taper(time=0.05)
    .pass_filter(time=(..., 300))
)


### **Exercise** (`Patch` processing 2)

Find some methods in the [processing module documentation](https://dascore.org/api/dascore/proc.html) and apply them to the event patch. Visualize the results. 

In [ ]:
# Exercise (Patch processing 2)


### Aggregate
[`Patch.aggregate`](https://dascore.org/api/dascore/proc/aggregate/aggregate.html) performs aggregations along specified dimension(s). Shortcuts for `min`, `max`, `mean`, etc. are also provided.

In [ ]:
# Use aggregate and mean to calculate the same patch.
mean_1 = patch.aggregate("time", method="mean")
mean_2 = patch.mean("time")

In [ ]:
# Aggregate supports any tensor reduction operation. 
reduced = patch.aggregate("time", np.sum)
print(reduced)

## Patch [Transformations](https://dascore.org/tutorial/transformations.html)
In addition to the processing methods, DASCore includes several patch [transformation methods](https://dascore.org/api/dascore/transform.html). This section shows some examples.

In [ ]:
# Load a cleaned-up patch from the previous example.
patch = dc.get_example_patch("example_event_2")

In [ ]:
# Perform a Discrete Fourier Transform (DFT) along the time axis. 
patch_fft = patch.dft("time", real=True)

In [ ]:
# Integrate along time domain
patch_int = patch.integrate("time")

In [ ]:
print(patch_fft), print(patch_int)

### **Exercise** (`Patch` Transformation)
Do the following to the patch above:

1. Perform a real discrete Fourier transform along the time axis
2. Get the amplitude spectra
3. Take the mean along the distance dimension
4. Select frequencies (`ft_time` dimension) less than 500 Hz
5. Plot using matplotlib and the `ft_time` dimension's values


In [ ]:
# Exercise (Patch Transformation)


## `Patch` [Conversion](https://dascore.org/recipes/external_conversion.html#obspy)

Although DASCore provides a lot of the functionality needed for DFOS processing, is not intended to do absolutely everything, and some other libraries might be better suited for a particular task. In addition to making it simple to access the underlying data as shown in the [patch tutorial](https://dascore.org/tutorial/patch.html), DASCore provides convenience functions to convert data to formats used by other libraries. 

In [ ]:

patch = dc.get_example_patch("example_event_1")

stream = patch.io.to_obspy()